In [1]:
""" This script tries only using the TuLiP toolbox to solve a multirobot motion planning problem.
Specifically, a field of cells and a number of robots are specified along with the system dynamics,
and constraints are written in monolithic LTL syntax.
The difference between this script and TuLiP_multirobot_solver.py
"""

# Import the packages that we need
from __future__ import print_function
import numpy as np
from tulip import spec
from tulip.transys import machines
from tulip import synth
import gr1_fragment
import State_encoding
import LTL_encoding

`omega.symbolic.symbolic` failed to import `dd.cudd`.
Will use `dd.autoref`.


In [2]:
# Workspace parameters
num_rows = 3
num_cols = 2
num_robots = 2
delay = 2
robot_init_pos = np.array([[0, 1], [1, 0]])
# The environment variables
env_vars = {}
env_init = set()
env_prog = set()
env_safe = set()

In [3]:
for r in range(1, num_robots + 1):
    env_vars[('x' + str(r))] = (0, num_cols - 1)
    env_vars[('y' + str(r))] = (0, num_rows - 1)
    env_vars[('delay' + str(r))] = 'boolean'
    env_vars[('d' + str(r))] = (-1, delay + 1)
print(env_vars)

{'x1': (0, 1), 'y1': (0, 2), 'delay1': 'boolean', 'd1': (-1, 3), 'x2': (0, 1), 'y2': (0, 2), 'delay2': 'boolean', 'd2': (-1, 3)}


In [4]:
for r in range(1, num_robots + 1):
    env_init |= {'x' + str(r) + '=' + str(robot_init_pos[r - 1][0])}
    env_init |= {'y' + str(r) + '=' + str(robot_init_pos[r - 1][1])}
    env_init |= {'d' + str(r) + '=0'}
    env_init |= {'!delay' + str(r)}
print(env_init)

{'!delay1', 'd2=0', '!delay2', 'y2=0', 'd1=0', 'x2=1', 'y1=1', 'x1=0'}


In [5]:
# Now no need to encode that each robot should only appear in one position
# Encode the transition relations
env_trans_x = set()
env_trans_y = set()
for r in range(1, num_robots + 1):
    for col in range(num_cols):
        if col != num_cols - 1:
            #env_trans_x |= {'( x' + str(r) + '=' + str(col) + ' && right' + str(r) + ' ) -> ( ( X(x' + str(r) + '=' +
            #                str(col+1) + ') && !delay' + str(r) + ' ) || ( X(x' + str(r) + '=' + str(col) +
            #                ') && delay' + str(r) + ' ) )'}
            env_trans_x |= {'( x' + str(r) + '=' + str(col) + ' && right' + str(r) + ' ) -> ( ( X(x' + str(r) + '=' +
                            str(col+1) + ' && !delay' + str(r) + ') ) || ( X(x' + str(r) + '=' + str(col) +
                            ' && delay' + str(r) + ') ) )'}
        if col != 0:
            env_trans_x |= {'( x' + str(r) + '=' + str(col) + ' && left' + str(r) + ' ) -> ( ( X(x' + str(r) + '=' +
                            str(col - 1) + ' && !delay' + str(r) + ') ) || ( X(x' + str(r) + '=' + str(col) +
                            ' && delay' + str(r) + ') ) )'}
        env_trans_x |= {'( x' + str(r) + '=' + str(col) + ' && up' + str(r) + ' ) -> X( x' + str(r) + '=' + str(col) +
                        ' )'}
        env_trans_x |= {'( x' + str(r) + '=' + str(col) + ' && down' + str(r) + ' ) -> X( x' + str(r) + '=' + str(col) +
                        ' )'}
        env_trans_x |= {'( x' + str(r) + '=' + str(col) + ' && stop' + str(r) + ' ) -> ( X( x' + str(r) + '=' + str(col) +
                        ' )' + ' )'}
    for row in range(num_rows):
        if row != num_rows - 1:
            env_trans_y |= {'( y' + str(r) + '=' + str(row) + ' && up' + str(r) + ' ) -> ( ( X(y' + str(r) + '=' +
                            str(row + 1) + ' && !delay' + str(r) + ') ) || ( X(y' + str(r) + '=' + str(row) +
                            ' && delay' + str(r) + ') ) )'}
        if row != 0:
            env_trans_y |= {'( y' + str(r) + '=' + str(row) + ' && down' + str(r) + ' ) -> ( ( X(y' + str(r) + '=' +
                            str(row - 1) + ' && !delay' + str(r) + ') ) || ( X(y' + str(r) + '=' + str(row) +
                            ' && delay' + str(r) + ') ) )'}
        env_trans_y |= {'( y' + str(r) + '=' + str(row) + ' && left' + str(r) + ' ) -> X( y' + str(r) + '=' + str(row) +
                        ' )'}
        env_trans_y |= {'( y' + str(r) + '=' + str(row) + ' && right' + str(r) + ' ) -> X( y' + str(r) + '=' + str(row) +
                        ' )'}
        env_trans_y |= {'( y' + str(r) + '=' + str(row) + ' && stop' + str(r) + ' ) -> ( X( y' + str(r) + '=' + str(row) +
                        ' )' + ' )'}
print(env_trans_x)
print(env_trans_y)
env_safe |= env_trans_x
env_safe |= env_trans_y

{'( x2=0 && right2 ) -> ( ( X(x2=1 && !delay2) ) || ( X(x2=0 && delay2) ) )', '( x2=0 && up2 ) -> X( x2=0 )', '( x2=1 && left2 ) -> ( ( X(x2=0 && !delay2) ) || ( X(x2=1 && delay2) ) )', '( x2=1 && down2 ) -> X( x2=1 )', '( x2=0 && stop2 ) -> ( X( x2=0 ) )', '( x2=1 && up2 ) -> X( x2=1 )', '( x1=0 && right1 ) -> ( ( X(x1=1 && !delay1) ) || ( X(x1=0 && delay1) ) )', '( x1=1 && stop1 ) -> ( X( x1=1 ) )', '( x2=1 && stop2 ) -> ( X( x2=1 ) )', '( x1=0 && stop1 ) -> ( X( x1=0 ) )', '( x1=1 && left1 ) -> ( ( X(x1=0 && !delay1) ) || ( X(x1=1 && delay1) ) )', '( x1=0 && up1 ) -> X( x1=0 )', '( x1=1 && up1 ) -> X( x1=1 )', '( x2=0 && down2 ) -> X( x2=0 )', '( x1=1 && down1 ) -> X( x1=1 )', '( x1=0 && down1 ) -> X( x1=0 )'}
{'( y2=2 && stop2 ) -> ( X( y2=2 ) )', '( y2=1 && stop2 ) -> ( X( y2=1 ) )', '( y1=1 && up1 ) -> ( ( X(y1=2 && !delay1) ) || ( X(y1=1 && delay1) ) )', '( y1=0 && right1 ) -> X( y1=0 )', '( y2=0 && right2 ) -> X( y2=0 )', '( y2=1 && right2 ) -> X( y2=1 )', '( y2=1 && down2 ) ->

In [6]:
# Relate the delay variables (delay#) to the delay-clock variables (d#)
delay2clock_minus = '( '
delay2clock_plus = ''
for r in range(1, num_robots + 1):
    delay2clock_minus += '( X(delay' + str(r) + ') && d' + str(r) + '=' + str(delay) + ' ) || '
delay2clock_plus = '( !' + delay2clock_minus[:-3] + ') ) -> ('
delay2clock_minus = delay2clock_minus[:-3] + ') -> ( '
for r in range(1, num_robots + 1):
    for d in range(delay + 1):
        delay2clock_minus += '( ( X(!delay' + str(r) + ') && d' + str(r) + '=' + str(d) + ' ) -> X(d' + str(r) + '=' + \
                             str(d - 1) + ') ) && '
        delay2clock_minus += '( ( X(delay' + str(r) + ') && d' + str(r) + '=' + str(d) + ' ) -> X(d' + str(r) + '=' + \
                             str(d) + ') ) && '
        delay2clock_plus += '( ( X(delay' + str(r) + ') && d' + str(r) + '=' + str(d) + ' ) -> X(d' + str(r) + '=' + \
                            str(d + 1) + ') ) && '
        delay2clock_plus += '( ( X(!delay' + str(r) + ') && d' + str(r) + '=' + str(d) + ' ) -> X(d' + str(r) + '=' + \
                            str(d) + ') ) && '
delay2clock_minus = delay2clock_minus[:-3] + ')'
delay2clock_plus = delay2clock_plus[:-3] + ')'
print(delay2clock_minus)
print(delay2clock_plus)
env_safe |= {delay2clock_minus}
env_safe |= {delay2clock_plus}

( ( X(delay1) && d1=2 ) || ( X(delay2) && d2=2 ) ) -> ( ( ( X(!delay1) && d1=0 ) -> X(d1=-1) ) && ( ( X(delay1) && d1=0 ) -> X(d1=0) ) && ( ( X(!delay1) && d1=1 ) -> X(d1=0) ) && ( ( X(delay1) && d1=1 ) -> X(d1=1) ) && ( ( X(!delay1) && d1=2 ) -> X(d1=1) ) && ( ( X(delay1) && d1=2 ) -> X(d1=2) ) && ( ( X(!delay2) && d2=0 ) -> X(d2=-1) ) && ( ( X(delay2) && d2=0 ) -> X(d2=0) ) && ( ( X(!delay2) && d2=1 ) -> X(d2=0) ) && ( ( X(delay2) && d2=1 ) -> X(d2=1) ) && ( ( X(!delay2) && d2=2 ) -> X(d2=1) ) && ( ( X(delay2) && d2=2 ) -> X(d2=2) ) )
( !( ( X(delay1) && d1=2 ) || ( X(delay2) && d2=2 ) ) ) -> (( ( X(delay1) && d1=0 ) -> X(d1=1) ) && ( ( X(!delay1) && d1=0 ) -> X(d1=0) ) && ( ( X(delay1) && d1=1 ) -> X(d1=2) ) && ( ( X(!delay1) && d1=1 ) -> X(d1=1) ) && ( ( X(delay1) && d1=2 ) -> X(d1=3) ) && ( ( X(!delay1) && d1=2 ) -> X(d1=2) ) && ( ( X(delay2) && d2=0 ) -> X(d2=1) ) && ( ( X(!delay2) && d2=0 ) -> X(d2=0) ) && ( ( X(delay2) && d2=1 ) -> X(d2=2) ) && ( ( X(!delay2) && d2=1 ) -> X(d2=

In [7]:
# Constraint on the maximum delay
env_max_delay = set()
env_not_delay = set()
for r in range(1, num_robots + 1):
    env_max_delay |= {'!(d' + str(r) + '=' + '-1)'}
    env_max_delay |= {'!(d' + str(r) + '=' + str(delay + 1) + ')'}
    env_not_delay |= {'!(delay' + str(r) + ')'}
print(env_max_delay)
print(env_not_delay)
env_safe |= env_max_delay
env_prog |= env_not_delay

{'!(d1=-1)', '!(d2=3)', '!(d1=3)', '!(d2=-1)'}
{'!(delay2)', '!(delay1)'}


In [8]:
# Define sys_vars
sys_vars = {}
for r in range(1, num_robots + 1):
    sys_vars['stop' + str(r)] = 'boolean'
    sys_vars['left' + str(r)] = 'boolean'
    sys_vars['right' + str(r)] = 'boolean'
    sys_vars['up' + str(r)] = 'boolean'
    sys_vars['down' + str(r)] = 'boolean'
print(sys_vars)

{'stop1': 'boolean', 'left1': 'boolean', 'right1': 'boolean', 'up1': 'boolean', 'down1': 'boolean', 'stop2': 'boolean', 'left2': 'boolean', 'right2': 'boolean', 'up2': 'boolean', 'down2': 'boolean'}


In [9]:
# Define initial command. Assume nothing.
sys_init = set()
sys_safe = set()
sys_prog = set()

In [10]:
# Avoid bad actions
sys_bad_act = set()
for r in range(1, num_robots + 1):
    sys_bad_act |= {'x' + str(r) + '=0 -> !left' + str(r)}
    sys_bad_act |= {'x' + str(r) + '=' + str(num_cols - 1) + ' -> !right' + str(r)}
    sys_bad_act |= {'y' + str(r) + '=0 -> !down' + str(r)}
    sys_bad_act |= {'y' + str(r) + '=' + str(num_rows - 1) + ' -> !up' + str(r)}
    # sys_bad_act |= {'delay' + str(r) + ' -> ' + '!stop' + str(r)}
print(sys_bad_act)
sys_safe |= sys_bad_act

{'x2=0 -> !left2', 'y1=2 -> !up1', 'y2=2 -> !up2', 'x2=1 -> !right2', 'x1=1 -> !right1', 'y2=0 -> !down2', 'y1=0 -> !down1', 'x1=0 -> !left1'}


In [11]:
# Encode the constraint that each time exactly one action must be chosen
sys_good_act = set()
for r in range(1, num_robots + 1):
    sys_good_act |= {'( left' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_good_act |= {'( right' + str(r) + ' ) <-> ( ' + '!left' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_good_act |= {'( up' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!left' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_good_act |= {'( down' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!left' +
                str(r) + ' && ' + '!stop' + str(r) + ' )'}
    sys_good_act |= {'( stop' + str(r) + ' ) <-> ( ' + '!right' + str(r) + ' && ' + '!up' + str(r) + ' && ' + '!down' +
                str(r) + ' && ' + '!left' + str(r) + ' )'}
sys_safe |= sys_good_act
print(sys_good_act)

{'( down2 ) <-> ( !right2 && !up2 && !left2 && !stop2 )', '( right1 ) <-> ( !left1 && !up1 && !down1 && !stop1 )', '( right2 ) <-> ( !left2 && !up2 && !down2 && !stop2 )', '( down1 ) <-> ( !right1 && !up1 && !left1 && !stop1 )', '( stop1 ) <-> ( !right1 && !up1 && !down1 && !left1 )', '( up2 ) <-> ( !right2 && !left2 && !down2 && !stop2 )', '( left1 ) <-> ( !right1 && !up1 && !down1 && !stop1 )', '( stop2 ) <-> ( !right2 && !up2 && !down2 && !left2 )', '( up1 ) <-> ( !right1 && !left1 && !down1 && !stop1 )', '( left2 ) <-> ( !right2 && !up2 && !down2 && !stop2 )'}


In [12]:
# Encode the constraint that if the last action was not executed yet, the same action should the next action
sys_same_act = set()
for r in range(1, num_robots + 1):
    sys_same_act |= {'( delay' + str(r) + ' && left' + str(r) + ' ) -> X( left' + str(r) + ' )'}
    sys_same_act |= {'( delay' + str(r) + ' && right' + str(r) + ' ) -> X( right' + str(r) + ' )'}
    sys_same_act |= {'( delay' + str(r) + ' && up' + str(r) + ' ) -> X( up' + str(r) + ' )'}
    sys_same_act |= {'( delay' + str(r) + ' && down' + str(r) + ' ) -> X( down' + str(r) + ' )'}
print(sys_same_act)
#sys_safe |= sys_same_act

{'( delay1 && up1 ) -> X( up1 )', '( delay1 && down1 ) -> X( down1 )', '( delay2 && down2 ) -> X( down2 )', '( delay2 && left2 ) -> X( left2 )', '( delay1 && right1 ) -> X( right1 )', '( delay1 && left1 ) -> X( left1 )', '( delay2 && up2 ) -> X( up2 )', '( delay2 && right2 ) -> X( right2 )'}


In [13]:
# User-defined system requirements
# Constraint 1: []<>([[0, 0], [0, 1], [0, 2]], 2)
# Constraint 2: []!([1, 1], 1)
# Constraint 3: ([]<>[1, 2], 2)

#sys_prog |= {'(x1=0)'}
#sys_safe |= {'!(x1=1 && y1=1)'}
sys_prog |= {'(x1=0) && (x2=0)'}
#sys_prog |= {'x1=1 && y1=0'}
#sys_prog |= {'x2=1 && y2=0'}
#sys_prog |= {'(x1=1) && (x2=1)'}
sys_safe |= {'!(x1=1 && y1=1) && !(x2=1 && y2=1)'}
sys_prog |= {'x1=1 && y1=2'}
sys_prog |= {'x2=1 && y2=2'}

In [14]:
# Create a GR(1) specification
specs = spec.GRSpec(env_vars, sys_vars, env_init, sys_init,
                    env_safe, sys_safe, env_prog, sys_prog)
specs.qinit = '\E \A'  # Moore initial condition synthesized too
specs.moore = False
specs.plus_one = False
print(specs)

((!delay1) && (d2=0) && (!delay2) && (y2=0) && (d1=0) && (x2=1) && (y1=1) && (x1=0) && [](( x2=1 && left2 ) -> ( ( X(x2=0 && !delay2) ) || ( X(x2=1 && delay2) ) )) && [](( x2=1 && down2 ) -> X( x2=1 )) && [](( y2=1 && stop2 ) -> ( X( y2=1 ) )) && [](( y1=1 && up1 ) -> ( ( X(y1=2 && !delay1) ) || ( X(y1=1 && delay1) ) )) && [](( x2=1 && up2 ) -> X( x2=1 )) && [](( y1=0 && right1 ) -> X( y1=0 )) && [](( y2=0 && right2 ) -> X( y2=0 )) && [](( y2=1 && right2 ) -> X( y2=1 )) && [](( x2=1 && stop2 ) -> ( X( x2=1 ) )) && [](( y1=0 && left1 ) -> X( y1=0 )) && [](( x1=0 && stop1 ) -> ( X( x1=0 ) )) && [](( y1=1 && right1 ) -> X( y1=1 )) && [](!(d1=-1)) && [](( y2=2 && right2 ) -> X( y2=2 )) && [](( ( X(delay1) && d1=2 ) || ( X(delay2) && d2=2 ) ) -> ( ( ( X(!delay1) && d1=0 ) -> X(d1=-1) ) && ( ( X(delay1) && d1=0 ) -> X(d1=0) ) && ( ( X(!delay1) && d1=1 ) -> X(d1=0) ) && ( ( X(delay1) && d1=1 ) -> X(d1=1) ) && ( ( X(!delay1) && d1=2 ) -> X(d1=1) ) && ( ( X(delay1) && d1=2 ) -> X(d1=2) ) && ( (

In [15]:
specs.check_syntax()

In [16]:
print('Start synthesis')
ctrl = synth.synthesize(specs)
print('End synthesis')
assert ctrl is not None, 'unrealizable'

Start synthesis
removed 21 nodes from 342 total
End synthesis


In [17]:
machines.random_run(ctrl, N=100)

move from
	 state: Sinit
	 with input:{'delay1': False, 'delay2': False, 'x1': 0, 'y1': 1, 'd1': 0, 'x2': 1, 'y2': 0, 'd2': 0}
	 to state: 0
	 reacting by producing output: {'stop1': False, 'left1': False, 'right1': False, 'up1': False, 'down1': True, 'stop2': False, 'left2': True, 'right2': False, 'up2': False, 'down2': False}
move from
	 state: 0
	 with input:{'delay1': False, 'delay2': True, 'x1': 0, 'y1': 0, 'd1': 0, 'x2': 1, 'y2': 0, 'd2': 1}
	 to state: 2
	 reacting by producing output: {'stop1': False, 'left1': False, 'right1': False, 'up1': True, 'down1': False, 'stop2': False, 'left2': True, 'right2': False, 'up2': False, 'down2': False}
move from
	 state: 2
	 with input:{'delay1': False, 'delay2': False, 'x1': 0, 'y1': 1, 'd1': 0, 'x2': 0, 'y2': 0, 'd2': 1}
	 to state: 278
	 reacting by producing output: {'stop1': False, 'left1': False, 'right1': False, 'up1': False, 'down1': True, 'stop2': False, 'left2': False, 'right2': False, 'up2': True, 'down2': False}
move from
	 state

([0,
  2,
  278,
  14,
  97,
  101,
  103,
  108,
  207,
  208,
  111,
  195,
  199,
  201,
  114,
  52,
  57,
  58,
  217,
  217,
  219,
  74,
  80,
  19,
  162,
  101,
  104,
  35,
  108,
  205,
  183,
  188,
  190,
  171,
  119,
  118,
  193,
  194,
  123,
  90,
  15,
  214,
  163,
  105,
  104,
  34,
  200,
  47,
  202,
  116,
  118,
  193,
  193,
  153,
  65,
  12,
  20,
  24,
  24,
  24,
  23,
  31,
  33,
  34,
  169,
  170,
  183,
  185,
  175,
  119,
  120,
  178,
  182,
  158,
  162,
  101,
  104,
  37,
  34,
  169,
  170,
  174,
  176,
  181,
  122,
  182,
  182,
  158,
  163,
  103,
  106,
  168,
  115,
  193,
  194,
  123,
  87,
  12,
  21,
  20],
 {'stop1': [False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   False,
   